# Vektoren und Matrizen
Erstellen Sie zwei Vektoren $a$ und $b$, die die Zahlen {0,2,4,6,8} enthalten, auf zwei Arten: einmal mit arange und einmal mit linspace. Setzen Sie dann den ersten Eintrag in $b$ auf 1 und geben Sie beide Vektoren aus. Berechnen Sie nun $\langle a,b \rangle$ und $M = a*b^T$ und geben Sie das Resultat und jeweils die resultierende Dimension aus (Achtung: zur Erstellung von $M$ müssen Sie die Vektoren $a$ und $b$ als Matrizen auffassen, z.B. mit np.asmatrix()).  Ersetzen Sie die Diagonale von $M$ nun so, dass die Determinante nicht 0 ist und lösen Sie dann das Gleichungssystem $Mx=b$.

In [1]:
import numpy as np
a = np.arange(0, 9, 2)
b = np.linspace(0, 8, 5)
b[0] = 1
x = a.dot(b)
# alternatively also with a.reshape(-1, 1).dot(b.reshape(1, -1))
M = np.outer(a, b)
print(x, f'dims = {x.shape}')
print(M, f'dims = {M.shape}')

120.0 dims = ()
[[ 0.  0.  0.  0.  0.]
 [ 2.  4.  8. 12. 16.]
 [ 4.  8. 16. 24. 32.]
 [ 6. 12. 24. 36. 48.]
 [ 8. 16. 32. 48. 64.]] dims = (5, 5)


In [2]:
M[np.diag_indices_from(M)] = 2
np.linalg.det(M)

-499808.0

In [3]:
np.linalg.solve(M, b)

array([0.5       , 0.11809335, 0.03374096, 0.02084   , 0.01523785])

# Matrix powers
a) Definieren Sie zunächst eine Funktion, die die Matrix
$$
  A_x = 
  \begin{pmatrix}
  1 - \cos(x)&\sin(x) \\
  \sin(x)&1+\cos(x) \\
  \end{pmatrix}
$$

für beliebige $x$ erstellt und zurückgibt. Schreiben Sie eine weitere Funktion `potenzen`, die $A^{n}_{x}$ numerisch für beliebige $n$ berechnet, einmal mithilfe von `matrix_power(A, power)` aus dem Untermodul `linalg` und einmal durch wiederholtes Anwenden von dot. Die Funktion soll beide Matrizen zurückgeben. Verwenden Sie nun den Befehl `np.finfo(np.float64).eps` um sich die Maschinengenauigkeit anzusehen mit welcher die Einträge abgespeichert sind.

In [4]:
import numpy as np
from IPython.display import display, Math
from decimal import Decimal
np.set_printoptions(precision=16)
def generate_A(x):
    A = np.array(
        [
            [1 - np.cos(x), np.sin(x)],
            [np.sin(x), 1 + np.cos(x)]
        ],
        dtype = Decimal
    )
    return A

def potenzen(M, n):
    M_pow_linalg = np.linalg.matrix_power(M, n)
    M_pow_dot = M.copy()
    for i in range(n - 1):
        M_pow_dot = M_pow_dot.dot(M.T)
    
    return M_pow_linalg, M_pow_dot

np.finfo(np.float64).eps

2.220446049250313e-16

`np.finfo(np.float64).eps` gibt die Differenz zwischen 1 und der nächstkleinsten repräsentierbaren Floatzahl zurück. Das bedeutet mit einem eps-Wert von 2.220446049250313e-16 ist die nächste repräsentierbare float von 1, 1.0000000000000002220446... was umgekehrt auch bedeutet, dass alle Differenzen zwischen floats die kleiner als dieser eps-Wert sind nicht mehr informativ ist und damit die verglichenen Zahlen für diese Präzision gleich sind. 

Vergleichen Sie dann für $x=6$ und $n=2,\ldots,30$ auf wieviele Nachkommastellen die beiden Berechnungen gleich sind (vergleichen Sie hierfür bis zur Maschinengenauigkeit).

In [5]:
def equal_decimal_places(a, b, precision = np.finfo(np.float64).eps):
    prec_lim_decimal_place = int(np.ceil(np.abs(np.log10(precision))))
    format_string = f'%.{prec_lim_decimal_place + 1}f'
    diff = np.abs(a - b)
    if  diff < precision:
        return prec_lim_decimal_place

    else:
        a_dec = (format_string % a).split('.')[-1][:-1]
        b_dec = (format_string % b).split('.')[-1][:-1]
        for i, (d1, d2) in enumerate(zip(a_dec, b_dec)):
            if d1 == d2:
                continue
            
            else:
                return i
            
def get_equal_decimal_places(m1, m2):
    decimal_places = []
    reshape_dim = m1.shape[0] * m1.shape[1]
    return [equal_decimal_places(val1, val2) for val1, val2 in zip(m1.reshape(reshape_dim), m2.reshape(reshape_dim))]

def format_latex(mat, matvar, x = '', n = '', label = ''):
    if label:
        latex_string = r'''
            %s^{%s}_{%s, %s} =
            \begin{pmatrix}
                %.16f & %.16f \\
                %.16f & %.16f \\
            \end{pmatrix}
        ''' % (matvar, n, x, label, *mat.reshape(4))
    
    else:
        latex_string = r'''
            %s^{%s}_{%s} =
            \begin{pmatrix}
                %.16f & %.16f \\
                %.16f & %.16f \\
            \end{pmatrix}
        ''' % (matvar, n, x, *mat.reshape(4))
    return latex_string

A = generate_A(6)
for n in np.arange(2, 31):
    for label, A_pow in zip(['matp', 'dotp'], potenzen(A, n)):
        display(
            Math(
                format_latex(A_pow, 'A', str(6), str(n), label)
            )
        )
    print(
        'equal decimal places for all values', 
        get_equal_decimal_places(
            *potenzen(A, n)
        ),
        '\n'
    )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

equal decimal places for all values [16, 16, 16, 16] 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

equal decimal places for all values [6, 5, 6, 5] 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

equal decimal places for all values [6, 5, 5, 5] 



Vergleichen Sie auch für weitere Werte von $x$.

b) Berechnen Sie $A^{n}_{6}$ für $n=1,\ldots,4$. Untersuchen sie nun genauso die Matrix $B_x=\frac{1}{2}A_x$, was fällt Ihnen dabei auf?

In [6]:
for n in np.arange(1, 5):
    for label, A_pow in zip(['matp', 'dotp'], potenzen(A, n)):
        display(
            Math(
                format_latex(A_pow, 'A', str(6), str(n), label)
            )
        )
    
    print()

B = 0.5*A
for n in np.arange(1, 5):
    for label, B_pow in zip(['matp', 'dotp'], potenzen(B, n)):
        display(
            Math(
                format_latex(B_pow, 'B', str(6), str(n), label)
            )
        )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Belegen Sie Ihre Vermutung indem Sie $B^{2}_{x}$ für beliebige $x$ berechnen (verwenden Sie hierfür sympy Routinen).

In [7]:
from sympy import Matrix
from sympy.abc import a, b, c, d
B = Matrix(
    [
        [a, b],
        [c, d]
    ]
)

for x in range(1, 11):
    latex_string_base = format_latex(
        np.asarray(
            B.evalf(subs = {k: v for k, v in zip('abcd', 0.5 * generate_A(x).reshape(4))})
        ),
        'B',
        str(x)
    )
    latex_string_pow = format_latex(
        np.asarray(
            (B*B).evalf(subs = {k: v for k, v in zip('abcd', 0.5 * generate_A(x).reshape(4))}), 
            dtype = np.float64
        ),
        'B',
        str(x),
        str(2)
    )
    
    for latex_string in [latex_string_base, latex_string_pow]:
        display(
            Math(
                latex_string
            )
        )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

c) Die besondere Struktur von $B_x$ können wir zur Evaluation der Methoden `dot` und `np.matrix_power` verwenden.  Rufen Sie Ihre Funktion `potenzen` für $B^{20}_{6}$ auf und vergleichen Sie die beiden Berechnungen mit $B$. Wählen Sie dafür die Ausgabegenauigkeit hoch genug (zB mit `np.set_printoptions(precision=16)`).

In [8]:
B = 0.5*generate_A(6)
np.set_printoptions(precision=16)
display(
    Math(
        format_latex(B, 'B', str(6))
    )
)

for label, B_pow in zip(['matp', 'dotp'], potenzen(B, 20)):
    display(
        Math(
            format_latex(B_pow, 'B', str(6), str(20), label)
        )
    )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Wiederholen Sie dies für mehrere Werte $x$ und stellen Sie eine Vermutung auf, welche der beiden Methoden exakter rechnet.

In [9]:
for x in np.arange(1, 16):
    B = 0.5*generate_A(x)
    for label, B_pow in zip(['matp', 'dotp'], potenzen(B, 20)):
        display(
            Math(
                format_latex(B_pow, 'B', str(x), str(20), label)
            )
        )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Evaluieren Sie dies nun, indem Sie für $n=2,\ldots,25$ die Norm von $(B_x−B^{n}_{x})$ für beide Methoden berechnen (im `numpy` Untermodul `linalg` gibt es die Funktion `norm`). Wie verhalten sich die beiden Methoden?

In [10]:
B = 0.5*generate_A(6)
for n in np.arange(2, 26):
    for label, B_pow in zip(['matp', 'dotp'], potenzen(B, n)):
        display(
            Math(
                r'||(B_{%i} - B^{%i}_{%i, %s})|| = {%.16f}' %
                (6, n, 6, label, np.linalg.norm(B - B_pow))
            )
        )

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

d) Mit den Untermodulen `Display` und `Latex` aus dem Modul `IPython.display` ist es möglich die Ausgaben mit LaTex-Quellcode zu formatieren. So können auch Laufvariablen integriert werden z.B.
```
for n in range(8, 15):
    display(
        Latex(f'$B^{ {n}}$:', B)
    )
```
Formatieren Sie dahingehend die Ausgabe Ihrer Lösungen in a) - c) übersichtlich. (Zusatzaufgabe: Vergleichen Sie was passiert, wenn in $A$ die Werte als Decimal anstatt float gespeichert werden.)

Die obigen Lösungen zeigen keinerlei Abweichung zwischen den Methoden für den default Datentyp `np.float64` allerdings sieht man einen deutlichen Unterschied zwischen der Matrixbasis und der Matrixpotenz im Falle des Typs `Decimal`, wobei sich hier zeigt dass die naive Implementierung über `dot` genauer zu sein scheint als `np.linalg.matrix_power`

# Function timing
Schreiben Sie ein Programm, das die Ausführungszeit der Methoden zur Eigenwertberechnung in sympy und numpy vergleicht. Erstellen Sie hierfür 2 Random-Matrizen von beliebiger Größe $n \times n$ mit Werten zwischen 0 und 1: benutzen Sie dafür `np.random` und die SymPy-Routine `randMatrix(size)`. Schreiben Sie dann zwei Funktionen `fnum` und `fsym`, welche die Eigenwerte der Matrizen mithilfe von `numpy` und `sympy` Methoden berechnen und zurückgeben. Die Eigenwerte sollen in beiden Fällen als numerische Werte dargestellt werden. Verwenden Sie jetzt die Methode `timeit(f,number=n)` aus dem gleichnamigen Modul `timeit` um die Zeit zu messen, die zur Ausführung benötigt wird (number gibt an wieviele Runden gemessen werden). Wie verhalten sich die Berechnungszeiten zueinander? Was passiert bei größeren Dimensionen der Matrizen?

In [11]:
import numpy as np
from sympy import randMatrix
from timeit import timeit

def time_numpy_eigvals(size, repeats = 1000):
    stmt = 'fnum(mat)'
    setup = \
    'import numpy as np\n' + \
    'def fnum(mat):\n' + \
    '\treturn np.linalg.eigvals(mat)\n' + \
    f'mat = np.random.random(size = ({size}, {size}))'
    return timeit(stmt, setup, number = repeats)

def time_sympy_eigvals(size, repeats = 1000):
    stmt = 'fsym(mat)'
    setup = \
    'from sympy import randMatrix\n' + \
    'def fsym(mat):\n' + \
    '\treturn [k.evalf() for k in mat.eigenvals().keys()]\n' + \
    f'mat = randMatrix({size}, min = 0, max = 100)/100'
    return timeit(stmt, setup, number = repeats)

In [12]:
time_numpy_eigvals(5, 100)

0.0031546480000006483

In [13]:
time_sympy_eigvals(5, 100)

1.1592778250000002

In [14]:
for n in range(2, 10):
    print(f'matrix size is {n} x {n}')
    print('numpy takes %.10f seconds to execute 100 times' % time_numpy_eigvals(n, 100))
    print('sympy takes %.10f seconds to execute 100 times' % time_sympy_eigvals(n, 100))

matrix size is 2 x 2
numpy takes 0.0017448840 seconds to execute 100 times
sympy takes 0.2302692460 seconds to execute 100 times
matrix size is 3 x 3
numpy takes 0.0020699650 seconds to execute 100 times
sympy takes 0.6163522760 seconds to execute 100 times
matrix size is 4 x 4
numpy takes 0.0025905310 seconds to execute 100 times
sympy takes 4.3695153690 seconds to execute 100 times
matrix size is 5 x 5
numpy takes 0.0028196620 seconds to execute 100 times
sympy takes 3.6831416880 seconds to execute 100 times
matrix size is 6 x 6
numpy takes 0.0029417590 seconds to execute 100 times
sympy takes 6.9899539850 seconds to execute 100 times
matrix size is 7 x 7
numpy takes 0.0030177500 seconds to execute 100 times
sympy takes 7.0813784480 seconds to execute 100 times
matrix size is 8 x 8
numpy takes 0.0031526520 seconds to execute 100 times
sympy takes 11.2024311650 seconds to execute 100 times
matrix size is 9 x 9
numpy takes 0.0030967980 seconds to execute 100 times
sympy takes 18.090258

Die Berechnungzeiten verhalten sich sehr stabil wobei es auch oft darauf ankommt was sonst noch so auf dem Rechner läuft. Sonderbar ist jedoch, das für große Matrizen das Timing für `sympy` doch länger dauert als jenes für `numpy` was sich jedoch nicht auf die Berechnungszeiten auswirkt.